Install the required packages. This code was run in Google Colab, so required reinstallation each time a new instance was started.

In [1]:
#Importing packages
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,173 kB]
Get:11 https://ppa.launchpadcontent.net

Import Packages - if running locally, code would start here after initial install

In [2]:
#Import Libraries
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.expected_conditions import element_to_be_clickable
from selenium.webdriver.common.keys import Keys
import pandas as pd
from time import sleep
from random import randint

set up chromedriver options

In [3]:
#set up driver options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

Webcrawling - collect individual book links

In [16]:
#Set up the driver and then run through each of the pages required to scrape the list of book links

#list of URLs - the next button on qbd does not have a unique name so the page is requested as
#a new URL instead of crawling through.
#create function
def get_linksqbd(urls):
  #start driver
  driver = webdriver.Chrome(options=chrome_options)
  wait = WebDriverWait(driver, timeout=45)
  #get the first url
  driver.get(urls[0])
  sleep(10)
  #extract page source and pass into a soup using BeautifulSoup
  sourceqbd = driver.page_source
  soupqbd = BeautifulSoup(sourceqbd)
  driver.quit()
  #run through remaining urls appending the soup from that page to the main one
  for url in urls[1:]:
    driver = webdriver.Chrome(options=chrome_options)
    wait = WebDriverWait(driver, timeout=45)
    driver.get(url)
    sleep(10)
    sourceqbd2 = driver.page_source
    soupqbd2 = BeautifulSoup(sourceqbd)
    soupqbd.append(soupqbd2)
    driver.quit()
  #return one long soup with all pages
  return soupqbd

In [8]:
childbesturls = ['https://www.qbd.com.au/childrens/best-sellers',
        'https://www.qbd.com.au/childrens/best-sellers/2/',
        ]

In [ ]:
nfbesturls = ['https://www.qbd.com.au/childrens/best-sellers',
        'https://www.qbd.com.au/childrens/best-sellers/2/',
        ]

In [ ]:
ficbesturls = ['https://www.qbd.com.au/fiction/best-sellers/',
               'https://www.qbd.com.au/fiction/best-sellers/2/',
               'https://www.qbd.com.au/fiction/best-sellers/3/']

In [ ]:
childgenurls = ['https://www.qbd.com.au/childrens/best-sellers/1500',
        'https://www.qbd.com.au/childrens/best-sellers/1501/',
        ]

In [17]:
childbestsoup = get_linksqbd(childbesturls)

In [18]:
qbdlinks = childbestsoup.find_all('div', class_="q2k21ptc")

In [23]:
#creat function to get the list of individual book links from a navigation page soup

def get_links(soup):
  #extract relevant divs
  Links = soup.find_all('div', class_="q2k21ptc")
  #empty list to append to
  links = []
  #pass through each row in the divs
  for row in Links:
    booklink = ''
    #find section of 'a'
    this = row.find('a')
    #try get the href (actual link) otherwise skip
    try:
      booklink = str(this.get('href')).strip()
      links.append(booklink)
      #print(booklink)
    except:
      pass
  #return list of links
  return links

In [24]:
qbd = get_links(childbestsoup)

In [25]:
qbd

['/dog-man-12-the-scarlet-shedder/dav-pilkey/9781338896435/',
 '/powerful-exclusive-cover/lauren-roberts/9781398536746/',
 '/powerless/lauren-roberts/9781398529489/',
 '/lgb-taylor-swift/wendy-loggia/9780593566718/',
 '/powerless-02-reckless/lauren-roberts/9781398530126/',
 '/detective-beans-and-the-case-of-the-missing-hat/li-chen/9781761049477/',
 '/diary-of-a-big-bad-wolf/ben-miller-elisa-paganelli/9781398533813/',
 '/if-only-i-had-told-her/laura-nowlin/9781728276229/',
 '/were-going-on-a-bear-hunt/michael-rosen-helen-oxenbury/9780744511352/',
 '/if-he-had-been-with-me/laura-nowlin/9781728205489/',
 '/the-day-i-fell-into-a-fairytale/ben-miller-daniela-jaglenka-terrazzini/9781471192449/',
 '/lightlark-01-lightlark/alex-aster/9781419760877/',
 '/artezans-01-the-forgotten-magic/ld-lapinski/9781510110090/',
 '/the-reappearance-of-rachel-price/holly-jackson/9780008507268/',
 '/a-good-girls-guide-to-murder/holly-jackson/9781405293181/',
 '/magicalia-01-race-of-wonders/jennifer-bell/9781529

In [22]:
len(qbd)

48